In [ ]:
import os
os.environ['AWS_DEFAULT_REGION'] = bucket_region
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key

import sagemaker

#Step 1: Model Creation
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",
    region=os.environ['AWS_DEFAULT_REGION'],
    version="1.0-1",
    py_version="py3",
    instance_type="ml.t3.medium",
)


import boto3
import time

# Set up credentials
ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID'] 
SECRET_KEY = os.environ['AWS_SECRET_ACCESS_KEY']
REGION = os.environ['AWS_DEFAULT_REGION'] 
# SESSION_TOKEN = 'YOUR_SESSION_TOKEN'  # This can be omitted if you're not using temporary credentials

# Initialize boto3 session
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name = REGION
    # aws_session_token=SESSION_TOKEN  # Omit this line if you're not using temporary credentials
)

# Create SageMaker, S3, and Runtime clients using the session
sagemaker_client = session.client('sagemaker')
s3_client = session.client('s3')
runtime_client = session.client('sagemaker-runtime')

# Step 2: Create a SageMaker Model
model_data_url = "s3://ai-gov-models/autoaimodel.tar.gz"
container = {
    "Image": image_uri,  # Replace with the correct URI
    "ModelDataUrl": model_data_url
}

model_name = "autoaimodel-v6"
response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn="arn:aws:iam::217560530765:role/service-role/AmazonSageMaker-ExecutionRole-20231024T083672",
    PrimaryContainer=container
)

# Step 3: Deploy the Model
config_name = "autoaimodel-endpoint-config-v6"
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=config_name,
    ProductionVariants=[
        {
            'InstanceType': 'ml.t3.medium',
            'InitialInstanceCount': 1,
            'ModelName': model_name,
            'VariantName': 'AllTraffic',
        }
    ]
)

endpoint_name = "autoaimodel-endpoint-v6"
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=config_name
)

# Wait until the endpoint is created
print("Waiting for endpoint to be in service...")
time.sleep(10*60)  # wait for 10 minutes, you might want to adjust this based on your actual deployment time

# Step 4: Make Inference Requests

# Load test data from the S3 location
response = s3_client.get_object(Bucket='ai-gov-models', Key='DemandResponseHoldout.csv')
test_data = response['Body'].read().decode('utf-8')  # assuming CSV format

# Just using the first row as an example, you can loop through each row for batch processing
sample_input = test_data.split("\n")[0]

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=sample_input,
    ContentType='text/csv'  # assuming CSV format for your model
)

prediction = response['Body'].read().decode()
print("Prediction:", prediction)

# Cleanup (Optional, but good practice to avoid charges)
response = sagemaker_client.delete_endpoint(EndpointName=endpoint_name)


Waiting for endpoint to be in service...
